### 필요한 라이브러리 설치

In [ ]:
!pip install pefile
!pip install numpy
!pip install sklearn
!pip install lightgbm
!pip install tqdm
!pip install pandas
!pip install graphviz
!pip install overload
!pip install lief

### 구글 드라이브 연동 및 데이터 다운로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/데이터.zip

### 사용할 헤더

In [ ]:
## Tutorial
import os
import glob
import json
import pprint

import numpy as np

from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

## PE-Miner
import pickle
# import  peminer
import seaborn as sns
# from overload import overload

## Ember
# import  ember
#import pandas as pd
import  pandas as  pd
from  tqdm.notebook import  tqdm
from  sklearn import  tree
from  sklearn.metrics import  precision_score
from  sklearn.metrics import  recall_score
from  sklearn.metrics import  f1_score
from  sklearn.metrics import  plot_confusion_matrix
from  sklearn.metrics import  confusion_matrix
from  sklearn.model_selection import  GridSearchCV
import  matplotlib.pyplot as  plt
from sklearn.feature_extraction import FeatureHasher

# from  pprint import  pprint

%matplotlib inline

### 파일 읽기

In [ ]:
def read_label_csv(path):
    label_table = dict()
    with open(path, "r", encoding = "cp949") as f: #csv파일을 읽는다
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r") as f:  #json파일을 읽는다 
        return json.load(f)

In [ ]:
learn_label_table = read_label_csv("학습데이터_정답.csv")
verify_label_table = read_label_csv("검증데이터_정답.csv")

###특징 벡터 생성 예시

*   PEMINER 정보는 모두 수치형 데이터이므로 특별히 가공을 하지 않고 사용 가능
*   EMBER, PESTUDIO 정보는 가공해서 사용해야 할 특징들이 있음 (e.g. imports, exports 등의 문자열 정보를 가지는 데이터)
*   수치형 데이터가 아닌 데이터(범주형 데이터)를 어떻게 가공할 지가 관건 >> 인코딩 (e.g. 원핫인코딩, 레이블인코딩 등)


> #### PEMINER



In [ ]:
## 중요도가 높은 상위 10개 단위로 가져온 리스트들
# peminer_header_10 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size']
# peminer_header_20 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData']
# peminer_header_30 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size']
peminer_header_40 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress']
# peminer_header_50 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics']
# peminer_header_60 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size']
# peminer_header_70 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL']
# peminer_header_80 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL']
# peminer_header_90 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers']
# peminer_header_100 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers']
## 중요도가 0인 것을 제외한 리스트(107)
# peminer_header_107 = ['COMCTL32.DLL', 'COMDLG32.DLL', 'CRTDLL.DLL', 'DCIMAN32.DLL', 'FileHeader.Characteristics', 'GDI32.DLL', 'LZ32.DLL', 'MAPI32.DLL', 'MPR.DLL', 'NETAPI32.DLL', 'OLEDLG.DLL', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.BaseOfCode', 'OptionalHeader.BaseOfData', 'OptionalHeader.CheckSum', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'OptionalHeader.FileAlignment', 'OptionalHeader.ImageBase', 'OptionalHeader.LoaderFlags', 'OptionalHeader.Magic', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.MajorLinkerVersion', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.MinorLinkerVersion', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'OptionalHeader.SectionAlignment', 'OptionalHeader.SizeOfCode', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.SizeOfHeapReserve', 'OptionalHeader.SizeOfImage', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.SizeOfStackReserve', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.Subsystem', 'RASAPI32.DLL', 'RPCRT4.DLL', 'SECUR32.DLL', 'SHELL32.DLL', 'SHLWAPI.DLL', 'Section.data.Characteristics', 'Section.data.NumberOfLinenumbers', 'Section.data.PointerToLinenumbers', 'Section.data.PointerToRawData', 'Section.data.PointerToRelocations', 'Section.data.SizeOfRawData', 'Section.data.VirtualAddress', 'Section.data.VirtualSize', 'Section.rsrc.Characteristics', 'Section.rsrc.NumberOfLinenumbers', 'Section.rsrc.NumberOfRelocations', 'Section.rsrc.PointerToLinenumbers', 'Section.rsrc.PointerToRawData', 'Section.rsrc.PointerToRelocations', 'Section.rsrc.SizeOfRawData', 'Section.rsrc.VirtualAddress', 'Section.rsrc.VirtualSize', 'Section.text.Characteristics', 'Section.text.NumberOfLinenumbers', 'Section.text.NumberOfRelocations', 'Section.text.PointerToLinenumbers', 'Section.text.PointerToRawData', 'Section.text.PointerToRelocations', 'Section.text.SizeOfRawData', 'Section.text.VirtualAddress', 'Section.text.VirtualSize', 'USER32.DLL', 'VERSION.DLL', 'WININET.DLL', 'WINMM.DLL', 'WINSOCK.DLL', 'WSOCK32.DLL']
# peminer_header_110 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL']
# peminer_header_120 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL']
# peminer_header_130 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL']
# peminer_header_140 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL']
# peminer_header_150 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL']
# peminer_header_160 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL']
# peminer_header_170 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL', 'RSRC32.DLL', 'Resource.Characteristics', 'Resource.MajorVersion', 'Resource.MinorVersion', 'Resource.NumberOfIdEntries', 'Resource.NumberOfNamedEntries', 'Resource.RT_ACCELERATOR', 'Resource.RT_BITMAP', 'Resource.RT_CURSOR', 'Resource.RT_DIALOG']
# peminer_header_180 = ['OptionalHeader.ImageBase', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'FileHeader.Characteristics', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.CheckSum', 'OptionalHeader.SizeOfImage', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'Section.rsrc.VirtualSize', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.Subsystem', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.SizeOfCode', 'Section.rsrc.SizeOfRawData', 'OptionalHeader.MajorLinkerVersion', 'Section.rsrc.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'Section.data.VirtualSize', 'Section.rsrc.PointerToRawData', 'Section.text.VirtualSize', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.BaseOfData', 'Section.data.PointerToRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'Section.text.SizeOfRawData', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'Section.data.VirtualAddress', 'Section.data.SizeOfRawData', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.MinorLinkerVersion', 'COMCTL32.DLL', 'OptionalHeader.SizeOfStackReserve', 'Section.text.PointerToRawData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'VERSION.DLL', 'Section.text.Characteristics', 'SHELL32.DLL', 'Section.rsrc.Characteristics', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'COMDLG32.DLL', 'OptionalHeader.BaseOfCode', 'Section.text.VirtualAddress', 'GDI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'Section.data.Characteristics', 'USER32.DLL', 'SHLWAPI.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'WSOCK32.DLL', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.FileAlignment', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'WINMM.DLL', 'WININET.DLL', 'MPR.DLL', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.SizeOfHeapReserve', 'SECUR32.DLL', 'OLEDLG.DLL', 'NETAPI32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.SectionAlignment', 'RPCRT4.DLL', 'RASAPI32.DLL', 'LZ32.DLL', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'CRTDLL.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'Section.rsrc.NumberOfLinenumbers', 'Section.text.PointerToLinenumbers', 'OptionalHeader.LoaderFlags', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.Magic', 'Section.text.PointerToRelocations', 'WINSOCK.DLL', 'Section.text.NumberOfLinenumbers', 'Section.rsrc.PointerToRelocations', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'Section.rsrc.PointerToLinenumbers', 'Section.text.NumberOfRelocations', 'Section.rsrc.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'MAPI32.DLL', 'Section.data.PointerToRelocations', 'Section.data.NumberOfLinenumbers', 'DCIMAN32.DLL', 'ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'DCIMAN.DLL', 'DSKMAINT.DLL', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'MAPI.DLL', 'MFC30.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL', 'RSRC32.DLL', 'Resource.Characteristics', 'Resource.MajorVersion', 'Resource.MinorVersion', 'Resource.NumberOfIdEntries', 'Resource.NumberOfNamedEntries', 'Resource.RT_ACCELERATOR', 'Resource.RT_BITMAP', 'Resource.RT_CURSOR', 'Resource.RT_DIALOG', 'Resource.RT_DLGINCLUDE', 'Resource.RT_FONT', 'Resource.RT_FONTDIR', 'Resource.RT_GROUP_CURSOR', 'Resource.RT_GROUP_ICON', 'Resource.RT_ICON', 'Resource.RT_MENU', 'Resource.RT_MESSAGETABLE', 'Resource.RT_RCDATA', 'Resource.RT_STRING']
## 원래 헤더(188)
# peminer_header = ['ADVAP132.DLL', 'AWFAXP32.DLL', 'AWFXAB32.DLL', 'AWPWD32.DLL', 'AWRESX32.DLL', 'AWUTIL32.DLL', 'BHNETB.DLL', 'BHSUPP.DLL', 'CCAPI.DLL', 'CCEI.DLL', 'CCPSH.DLL', 'CCTN20.DLL', 'CMC.DLL', 'COMCTL32.DLL', 'COMDLG32.DLL', 'CRTDLL.DLL', 'DCIMAN.DLL', 'DCIMAN32.DLL', 'DSKMAINT.DLL', 'FileHeader.Characteristics', 'FileHeader.Machine', 'FileHeader.NumberOfSections', 'FileHeader.NumberOfSymbols', 'FileHeader.PointerToSymbolTable', 'FileHeader.SizeOfOptionalHeader', 'FileHeader.TimeDateStamp', 'GDI32.DLL', 'GROUP.DLL', 'HYPERTERM.DLL', 'KERNL32.DLL', 'LZ32.DLL', 'MAPI.DLL', 'MAPI32.DLL', 'MFC30.DLL', 'MPR.DLL', 'MSFS32.DLL', 'MSNDUI.DLL', 'MSNET32.DLL', 'MSPST32.DLL', 'MSSHRUI.DLL', 'MSVIEWUT.DLL', 'NAL.DLL', 'NDIS30.DLL', 'NETAPI.DLL', 'NETAPI32.DLL', 'NETBIOS.DLL', 'NETDI.DLL', 'NETSETUP.DLL', 'NWAB32.DLL', 'NWNET32.DLL', 'NWNP32.DLL', 'OLEDLG.DLL', 'OptionalHeader.AddressOfEntryPoint', 'OptionalHeader.BaseOfCode', 'OptionalHeader.BaseOfData', 'OptionalHeader.CheckSum', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BASERELOC.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DEBUG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXCEPTION.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_EXPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_GLOBALPTR.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IAT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_IMPORT.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESERVED.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_RESOURCE.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_SECURITY.VirtualAddress', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.Size', 'OptionalHeader.DataDirectory.IMAGE_DIRECTORY_ENTRY_TLS.VirtualAddress', 'OptionalHeader.DllCharacteristics', 'OptionalHeader.FileAlignment', 'OptionalHeader.ImageBase', 'OptionalHeader.LoaderFlags', 'OptionalHeader.Magic', 'OptionalHeader.MajorImageVersion', 'OptionalHeader.MajorLinkerVersion', 'OptionalHeader.MajorOperatingSystemVersion', 'OptionalHeader.MajorSubsystemVersion', 'OptionalHeader.MinorImageVersion', 'OptionalHeader.MinorLinkerVersion', 'OptionalHeader.MinorOperatingSystemVersion', 'OptionalHeader.MinorSubsystemVersion', 'OptionalHeader.NumberOfRvaAndSizes', 'OptionalHeader.Reserved1', 'OptionalHeader.SectionAlignment', 'OptionalHeader.SizeOfCode', 'OptionalHeader.SizeOfHeaders', 'OptionalHeader.SizeOfHeapCommit', 'OptionalHeader.SizeOfHeapReserve', 'OptionalHeader.SizeOfImage', 'OptionalHeader.SizeOfInitializedData', 'OptionalHeader.SizeOfStackCommit', 'OptionalHeader.SizeOfStackReserve', 'OptionalHeader.SizeOfUninitializedData', 'OptionalHeader.Subsystem', 'POWERCFG.DLL', 'RASAPI16.DLL', 'RASAPI32.DLL', 'RASPI.DLL', 'RPCLTC1.DLL', 'RPCNS4.DLL', 'RPCRT4.DLL', 'RPCTLC3.DLL', 'RPCTLC5.DLL', 'RPCTLC6.DLL', 'RPCTLS3.DLL', 'RPCTLS5.DLL', 'RPCTLS6.DLL', 'RSRC32.DLL', 'Resource.Characteristics', 'Resource.MajorVersion', 'Resource.MinorVersion', 'Resource.NumberOfIdEntries', 'Resource.NumberOfNamedEntries', 'Resource.RT_ACCELERATOR', 'Resource.RT_BITMAP', 'Resource.RT_CURSOR', 'Resource.RT_DIALOG', 'Resource.RT_DLGINCLUDE', 'Resource.RT_FONT', 'Resource.RT_FONTDIR', 'Resource.RT_GROUP_CURSOR', 'Resource.RT_GROUP_ICON', 'Resource.RT_ICON', 'Resource.RT_MENU', 'Resource.RT_MESSAGETABLE', 'Resource.RT_RCDATA', 'Resource.RT_STRING', 'Resource.RT_VERSION', 'Resource.TimeDateStamp', 'SAPNSP.DLL', 'SECUR32.DLL', 'SHELL32.DLL', 'SHLWAPI.DLL', 'SLENH.DLL', 'Section.data.Characteristics', 'Section.data.NumberOfLinenumbers', 'Section.data.NumberOfRelocations', 'Section.data.PointerToLinenumbers', 'Section.data.PointerToRawData', 'Section.data.PointerToRelocations', 'Section.data.SizeOfRawData', 'Section.data.VirtualAddress', 'Section.data.VirtualSize', 'Section.rsrc.Characteristics', 'Section.rsrc.NumberOfLinenumbers', 'Section.rsrc.NumberOfRelocations', 'Section.rsrc.PointerToLinenumbers', 'Section.rsrc.PointerToRawData', 'Section.rsrc.PointerToRelocations', 'Section.rsrc.SizeOfRawData', 'Section.rsrc.VirtualAddress', 'Section.rsrc.VirtualSize', 'Section.text.Characteristics', 'Section.text.NumberOfLinenumbers', 'Section.text.NumberOfRelocations', 'Section.text.PointerToLinenumbers', 'Section.text.PointerToRawData', 'Section.text.PointerToRelocations', 'Section.text.SizeOfRawData', 'Section.text.VirtualAddress', 'Section.text.VirtualSize', 'UMDM32.DLL', 'USER32.DLL', 'VERSION.DLL', 'WININET.DLL', 'WINMM.DLL', 'WINREG.DLL', 'WINSOCK.DLL', 'WS2.DLL', 'WSOCK32.DLL']

class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        # 원래 코드
        # self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        # 바뀐 코드
        for _,value in sorted(self.report.items(), key=lambda x: x[0]):
          if _ in peminer_header_40:
            self.vector.append(value)
        return self.vector

class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        # 원래 코드
        # self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        # 바뀐 코드
        for _,value in sorted(self.report.items(), key=lambda x: x[0]):
          if _ in peminer_header_40:
            self.vector.append(value)
        return self.vector



> #### EMBER



In [ ]:
class EmberParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()
    
    def get_byteentropy_info(self):
        byteentropy = np.array(self.report["byteentropy"])
        total = byteentropy.sum()
        vector = byteentropy / total
        return vector.tolist()

    def get_string_info(self):
        strings = self.report["strings"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            strings['paths'], 
            strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
             general['exports'], general['has_debug'], general['has_relocations'], general['has_resources'],
            general['has_signature'], general['has_tls'], general['size']
        ]
        return vector

    def get_imports_info(self):
        imports = self.report["imports"]
        libraries = list(set([l.lower() for l in imports.keys()]))
        libraries_hashed = FeatureHasher(256, input_type="string").transform([libraries]).toarray()[0]
        import_ = [lib.lower() + ':' + e for lib, elist in imports.items() for e in elist]
        imports_hashed = FeatureHasher(1024, input_type="string").transform([import_]).toarray()[0]
        vector = np.hstack([libraries_hashed, imports_hashed])
        return vector.tolist()

    def get_datadirectories_info(self):
        # 특정 데이터만 가져오기 위해 중요도가 높은 데이터의 index만 Data_Directory 리스트에 저장
        Data_Directory = [0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 19, 20, 21, 24, 25, 27, 29]
        datadirectories = self.report["datadirectories"]
        name_order = ["EXPORT_TABLE", "IMPORT_TABLE", "RESOURCE_TABLE", "EXCEPTION_TABLE", "CERTIFICATE_TABLE",
                      "BASE_RELOCATION_TABLE", "DEBUG", "ARCHITECTURE", "GLOBAL_PTR", "TLS_TABLE", "LOAD_CONFIG_TABLE",
                      "BOUND_IMPORT", "IAT", "DELAY_IMPORT_DESCRIPTOR", "CLR_RUNTIME_HEADER"]
        vector = np.zeros(19, dtype=np.float32)
        cnt = 0
        for i in Data_Directory:
          if i < len(datadirectories):
              if i%2 == 0:
                vector[cnt] = datadirectories[i//2]["size"]
              if i%2 == 1:
                vector[cnt] = datadirectories[i//2]["virtual_address"]
          cnt += 1
        return vector.tolist()

    def get_section_info(self):
        # 특정 데이터만 가져오기 위해 중요도가 높은 데이터의 index만 아래의 리스트에 저장      
        Section_General = [1, 3, 4]
        Section_Size = [1, 4, 5, 14, 22, 28, 32, 35, 41, 42, 43]
        Section_Entropy = [5, 14, 20, 28, 32, 35, 40, 41, 42, 43]
        Section_Virtual_Size = [0, 4, 5, 10, 14, 16, 27, 32, 34, 35, 38, 41, 42, 43]
        Section_Entry = [9, 17, 20, 39]
        Section_Characteristic = [13, 23, 37, 47]

        sections = self.report["section"]
        vector = [
            #0. len(sections),  # total number of sections
            sum(1 for s in sections['sections'] if s['size'] == 0), # number of sections with nonzero size
            #2. sum(1 for s in sections['sections'] if s['name'] == ""),  # number of sections with an empty name
            sum(1 for s in sections['sections'] if 'MEM_READ' in s['props'] and 'MEM_EXECUTE' in s['props']), # number of RX
            sum(1 for s in sections['sections'] if 'MEM_WRITE' in s['props']) # number of W
        ]
        # gross characteristics of each section
        section_sizes = [(s['name'], s['size']) for s in sections['sections']]
        section_sizes_hashed = FeatureHasher(50, input_type="pair").transform([section_sizes]).toarray()[0] # 50 
        section_sizes_hashed_new = []
        for value in Section_Size:
          section_sizes_hashed_new.append(section_sizes_hashed[value])
        
        section_entropy = [(s['name'], s['entropy']) for s in sections['sections']]
        section_entropy_hashed = FeatureHasher(50, input_type="pair").transform([section_entropy]).toarray()[0] # 50
        section_entropy_hashed_new = []
        for value in Section_Entropy:
          section_entropy_hashed_new.append(section_entropy_hashed[value])

        section_vsize = [(s['name'], s['vsize']) for s in sections['sections']]
        section_vsize_hashed = FeatureHasher(50, input_type="pair").transform([section_vsize]).toarray()[0]
        section_vsize_hashed_new = []
        for value in Section_Virtual_Size:
          section_vsize_hashed_new.append(section_vsize_hashed[value])
        
        entry_name_hashed = FeatureHasher(50, input_type="string").transform([sections['entry']]).toarray()[0]
        entry_name_hashed_new = []
        for value in Section_Entry:
          entry_name_hashed_new.append(entry_name_hashed[value])

        characteristics = [p for s in sections['sections'] for p in s['props'] if s['name'] == sections['entry']]
        characteristics_hashed = FeatureHasher(50, input_type="string").transform([characteristics]).toarray()[0]
        characteristics_hashed_new = []
        for value in Section_Characteristic:
          characteristics_hashed_new.append(characteristics_hashed[value])
        
        hstack = np.hstack([
            vector, section_sizes_hashed_new, section_entropy_hashed_new, section_vsize_hashed_new, entry_name_hashed_new,
            characteristics_hashed_new
        ])
        return hstack.tolist()

    def get_exports_info(self):
        exports_hashed = FeatureHasher(128, input_type="string").transform([self.report]).toarray()[0]
        return exports_hashed.tolist()

    def get_header_file_info(self):
        # 특정 데이터만 가져오기 위해 중요도가 높은 데이터의 index만 Header_Info 리스트에 저장      
        Header_Info = [0, 11, 14, 15, 28, 29, 34, 51, 52, 53, 54, 55, 57, 59, 60]
        fh1 = FeatureHasher(10, input_type="string").transform([self.report['header']['coff']['characteristics']]).toarray()[0]
        fh1_new = []
        for i in [0,3,4]:
          fh1_new.append(fh1[i])
        fh2 = FeatureHasher(10, input_type="string").transform([[self.report['header']['optional']['subsystem']]]).toarray()[0]  # 21-30 -> 28, 29
        fh2_new = []
        for i in [7,8]:
          fh2_new.append(fh2[i])
        fh3 = FeatureHasher(10, input_type="string").transform([self.report['header']['optional']['dll_characteristics']]).toarray()[0]  # 31-40 ->34
        fh3_new = []
        fh3_new.append(fh3[3])

        # 리스트에 해당되지 않은 데이터들은 가져올 필요가 없으므로 주석 처리
        hstack = np.hstack([
            self.report['header']['coff']['timestamp'], # 0 
            #FeatureHasher(10, input_type="string").transform([[self.report['header']['coff']['machine']]]).toarray()[0],  # 1~10 
            fh1_new, # 11-20 -> 11,14,15
            fh2_new,  # 21-30 -> 28, 29
            fh3_new,  # 31-40 ->34
            #FeatureHasher(10, input_type="string").transform([[self.report['header']['optional']['magic']]]).toarray()[0],  # 41-50
            self.report['header']['optional']['major_image_version'], # 51
            self.report['header']['optional']['minor_image_version'], # 52
            self.report['header']['optional']['major_linker_version'],# 53
            self.report['header']['optional']['minor_linker_version'],# 54
            self.report['header']['optional']['major_operating_system_version'],# 55
            #self.report['header']['optional']['minor_operating_system_version'],# 56
            self.report['header']['optional']['major_subsystem_version'],# 57
            #self.report['header']['optional']['minor_subsystem_version'],# 58
            self.report['header']['optional']['sizeof_code'],# 59
            self.report['header']['optional']['sizeof_headers'],# 60
            #self.report['header']['optional']['sizeof_heap_commit'],# 61
        ])
        return hstack.tolist()


    def process_report(self):
        vector = []
        vector += self.get_general_file_info()
        #vector += self.get_histogram_info()
        #vector += self.get_byteentropy_info()
        #vector += self.get_string_info()
        vector += self.get_imports_info()
        vector += self.get_datadirectories_info()
        vector += self.get_section_info()
        vector += self.get_exports_info()
        vector += self.get_header_file_info()

        return vector


> #### PESTUDIO



In [ ]:
class PestudioParser:
    def __init__(self, path, flag = 0):
        if flag == -1:
          return
        self.report = read_json(path)
        self.vector = []

    def get_max_entropy_info(self, flag = 0):
        entropy = []
        try:
            if flag == -1:
              raise Exception
            if "entropy" in self.report["image"]["dos-header"]:
              entropy.append(float(self.report["image"]["dos-header"]["entropy"]))
            if "entropy" in self.report["image"]["dos-stub"]:
              entropy.append(float(self.report["image"]["dos-stub"]["entropy"]))
            if "entropy" in self.report["image"]["overlay"]:
              entropy.append(float(self.report["image"]["overlay"]["entropy"]))
            if "entropy" in self.report["image"]["overview"]:
              entropy.append(float(self.report["image"]["overview"]["entropy"]))
            
            Max = max(entropy)

            if Max >= 6.0:
                result = 1
            else:
                result = 0
        except Exception:
            result = -1
        return [result]
       
    def get_certificate_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            certificate = self.report["image"]["certificate"]

            if certificate == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]   

    def get_debug_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            debug = self.report["image"]["debug"]
            if debug == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]        

    def get_network_run_from_swap_info(self, flag = 0):
        try:
            if flag == -1:
              raise Exception
            network_run_from_swap = self.report["image"]["file-header"]["network-run-from-swap"]
            if network_run_from_swap == "true":
                result = 1
            else:
                result = 0
        except: 
            result = -1
        return [result]

    def get_control_flow_guard_info(self, flag = 0):
        try:
            if flag == -1:
              raise Exception
            control_flow_guard = self.report["image"]["optional-header"]["control-flow-guard"]
            if control_flow_guard == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]

    def get_exports_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            exports = self.report["image"]["exports"]
            if exports == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]

    def get_tls_callbacks_info(self, flag = 0):
        try: 
            if flag == -1:
              raise Exception
            tls_callbacks = self.report["image"]["tls-callbacks"]
            if tls_callbacks == "n/a":
                result = 1
            else:
                result = 0
        except:
            result = -1
        return [result]

    def get_section_info(self, flag = 0):
        try:
            if flag == -1:
              raise Exception
            sections = self.report["image"]["sections"]["section"]
            num_of_section = len(section)

            max_entropy = 0.0
            min_entropy = 50.0
            excutable = 0
            for section in sections:
                max_entropy = max(max_entropy, float(section["@entropy"]))
                min_entropy = min(min_entropy, float(section["@entropy"]))        

                if section["@excutable"] == "-":
                    excutable += 1
                else:
                    excutable += 0
        except:
            num_of_section = 0
            max_entropy = -1.0
            min_entropy = -1.0
            excutable = -1
        return [num_of_section, max_entropy, min_entropy, excutable]

    def get_import_info(self, flag = 0):
        dic_o = {'VerQueryValueA': 3826, 'GetFileVersionInfoA': 3381, 'GetFileVersionInfoSizeA': 3376, 'lstrcmpiA': 3359, 'HeapReAlloc': 3285, 'FindNextFileA': 3272, 'SetErrorMode': 3241, 'DispatchMessageA': 3241, 'IsWindowEnabled': 3200}
        vector = []
        try:
            if flag == -1:
              raise Exception
            imports = self.report["image"]["imports"]["import"]
            for key,value in dic_o.items():
              # print(key)
              if key in imports["@name"]:
                vector.append(1)
              else:
                vector.append(0)
        except:
            vector = [-1 for i in range(len(dic_o))]
        return vector

    def get_import_info2(self, flag = 0):
        dic_x = {'': 159559, 'RegQueryValueExA': 5448, 'RegOpenKeyExA': 5358, 'SelectObject': 5280, 'GetDeviceCaps': 5019, 'SetTextColor': 4967, 'CharNextA': 4890, 'SetBkColor': 4879, 'FindFirstFileA': 4766, 'SetBkMode': 4735, 'DestroyWindow': 4552, 'GetStartupInfoA': 4496, 'GetFileType': 4437, 'lstrcpynA': 4427, 'ImageList_Create': 4411}
        vector = []
        try:
            if flag == -1:
              raise Exception
            imports = self.report["image"]["imports"]["import"]
            for key,value in dic_x.items():
              # print(key)
              if key in imports["@name"]:
                vector.append(1)
              else:
                vector.append(0)
        except:
            vector = [-1 for i in range(len(dic_x))]
        return vector      
            
    def get_virustotal_info(self, flag = 0):  # 정상 악성 비교해봤을 때, 악성은 offline이 아님
        try:
            if flag == -1:
              raise Exception
            if self.report["image"]["virustotal"] != "offline":
                result = 1
            else:
                result = 0
        except:
            result = -1

        return [result]
        
    def process_report(self, flag = 0):
        vector = []
        vector += self.get_max_entropy_info(flag)
        vector += self.get_certificate_info(flag)
        vector += self.get_debug_info(flag)
        vector += self.get_network_run_from_swap_info(flag)
        vector += self.get_control_flow_guard_info(flag)
        vector += self.get_exports_info(flag)
        vector += self.get_tls_callbacks_info(flag)
        vector += self.get_section_info(flag)
        vector += self.get_import_info(flag)
        vector += self.get_import_info2(flag)
        vector += self.get_virustotal_info(flag)
        return vector

### 학습데이터 구성

> #### PEMINER, EMBER

In [ ]:
import os
import queue

learn_data = []

def get_subdir(path):
    global learn_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            learn_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PEMINER/학습데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

#for dir in learn_data:
#    print(dir)

> #### PESTUDIO



In [ ]:
import os
import queue

pestudio_data = []

def get_subdir(path):
    global pestudio_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            pestudio_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PESTUDIO/학습데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

#for dir in pestudio_data:
#    print(dir)

> #### PEMINER + EMBER + PESTUDIO

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X
# 데이터의 레이블 모음(1차원 리스트) : y
X, y = [], []

for path in learn_data:
  feature_vector = []
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  label = learn_label_table[fname]
  for data in ["PEMINER", "EMBER","PESTUDIO"]:
        path = f"{data}/{fdata}/{fname}.json"
        if data == "PEMINER":
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO" and path in pestudio_data:
            feature_vector += PestudioParser(path).process_report()
        else:  
            feature_vector += PestudioParser(path, -1).process_report(-1)
  X.append(feature_vector)
  y.append(label)

np.asarray(X).shape, np.asarray(y).shape

((20000, 1571), (20000,))

### 검증데이터 구성

> #### PEMINER, EMBER

In [ ]:
import os
import queue

veri_data = []

def get_subdir(path):
    global veri_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            veri_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PEMINER/검증데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)




> #### PESTUDIO



In [ ]:
import os
import queue

pestudio_veri_data = []

def get_subdir(path):
    global pestudio_veri_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            pestudio_veri_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list

dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PESTUDIO/검증데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

> #### PEMINER + EMBER + PESTUDIO

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : X
# 데이터의 레이블 모음(1차원 리스트) : y
A, b = [], []

for path in veri_data:
  feature_vector = []
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  label = verify_label_table[fname]
  for data in ["PEMINER", "EMBER","PESTUDIO"]:
        path = f"{data}/{fdata}/{fname}.json"
        if data == "PEMINER":
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO" and path in pestudio_veri_data:
            feature_vector += PestudioParser(path).process_report()
        else:  
            feature_vector += PestudioParser(path, -1).process_report(-1)
  A.append(feature_vector)
  b.append(label)

np.asarray(A).shape, np.asarray(b).shape

((10000, 1571), (10000,))

### 학습 및 검증

In [ ]:
SEED = 41

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt": # decision tree
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf

def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print("정확도", model.score(X_test, y_test))


In [ ]:
# 학습
models = []
for model in ["rf", "lgb"]:
    clf = train(X, y, model)
    models.append(clf)

# 검증
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함
for model in models:
    evaluate(A, b, model)

정확도 0.9547
정확도 0.9563


### 앙상블

In [ ]:
def ensemble_result(X, y, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
   
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''
    
    # Soft Voting
    # https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]  
    print("정확도", accuracy_score(y, predict))

In [ ]:
ensemble_result(A, b, models)

정확도 0.9595


### 테스트데이터 구성

> #### PEMINER, EMBER - Test Data

In [ ]:
import os
import queue

test_data = []

def get_subdir(path):
    global test_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            test_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PEMINER/테스트데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

> #### PESTUDIO - Test Data

In [ ]:
import os
import queue

pestudio_test_data = []

def get_subdir(path):
    global pestudio_test_data
    try:  # 검색이 허가되지 않은 디렉토리 접근에 관한 예외처리
        dirfiles = os.listdir(path)  # path에 해당하는 디렉토리 dirfiles에 추가
    except PermissionError:
        return []

    subdir_list = []
    for each in dirfiles:
        if each.endswith(".json"):  # .txt로 끝나는 파일은 따로 all_txt리스트에 저장
            pestudio_test_data.append(path + each)
        full_name = path + each  # path와 each(디렉토리)
        if os.path.isdir(full_name):
            subdir_list.append(full_name + "/")
    return subdir_list


dir_queue = queue.Queue()
# /Library/ 아래의 모든 하위 디렉토리 찾기
dir_queue.put("PESTUDIO/테스트데이터/")  # 검색하고자 하는 디렉토리를 줄 앞에 세움

while not dir_queue.empty():  # 큐가 비어있는 상태인지 확인
    dir_name = dir_queue.get()  # 큐에서 빼낸 dir_name
    subdir_names = get_subdir(dir_name)  # dir_name의 하위 디렉토리를 subdir_names에 저장
    for each in subdir_names:  # subdir_names리스트에 있는 디렉토리들을 다시 큐에 넣음
        dir_queue.put(each)

> #### PEMINER + EMBER + PESTUDIO - Test Data

In [ ]:
# 데이터의 특징 벡터 모음(2차원 리스트) : T
T = []
file_name = []
for path in test_data:
  global file_name
  feature_vector = []
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  file_name.append(fname)
  for data in ["PEMINER", "EMBER", "PESTUDIO"]:
        path = f"{data}/{fdata}/{fname}.json"
        if data == "PEMINER":
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO" and path in pestudio_veri_data:
            feature_vector += PestudioParser(path).process_report()
        else:  
            feature_vector += PestudioParser(path, -1).process_report(-1)
  T.append(feature_vector)

np.asarray(T).shape

(10000, 1571)

### 테스트 실행

In [ ]:
def test(X, models):
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]

    return predict    

test_ret = test(T, models)

import csv

f = open('predict.csv', 'w', newline='')
wr = csv.writer(f)
wr.writerow(['file', 'predict'])

for i in range(len(test_ret)):
  wr.writerow([file_name[i], test_ret[i]])

f.close()

---
### PESTUDIO import 함수에서 사용할 리스트 생성
- 주어진 모든 데이터에 대해 함수이름 저장
1.   정상파일 함수 이름 & 빈도수 : func_name_o
2.   악성파일 함수 이름 & 빈도수 : func_name_x

In [ ]:
func_name = {}
for path in pestudio_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = learn_label_table[fname]
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name:
      func_name[name] = 1
    else:
      func_name[name] += 1

In [ ]:
for path in pestudio_veri_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = verify_label_table[fname]
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name:
      func_name[name] = 1
    else:
      func_name[name] += 1

In [ ]:
func_name_o = {}
for path in pestudio_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = learn_label_table[fname]
  if label == 1: # 악성파일인 경우 pass
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_o:
      func_name_o[name] = 1
    else:
      func_name_o[name] += 1

In [ ]:
for path in pestudio_veri_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = verify_label_table[fname]
  if label == 1: # 악성파일인 경우 pass
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_o:
      func_name_o[name] = 1
    else:
      func_name_o[name] += 1

In [ ]:
func_name_x = {}
for path in pestudio_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = learn_label_table[fname]
  if label == 0:
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_x:
      func_name_x[name] = 1
    else:
      func_name_x[name] += 1

In [ ]:
for path in pestudio_veri_data:
  temp, fdata, fname = path.strip().split("/")
  fname = fname.replace(".json","")
  path = f"PESTUDIO/{fdata}/{fname}.json"
  data = read_json(path)
  label = verify_label_table[fname]
  if label == 0:
    continue
  if "image" not in data:
    continue
  if "imports" not in data["image"]:
    continue
  if "import" not in data["image"]["imports"]:
    continue
  for pe_import in data["image"]["imports"]["import"]:
    if "@name" not in pe_import:
      continue
    if pe_import == "@name":
      continue
    name = pe_import["@name"]
    if name not in func_name_x:
      func_name_x[name] = 1
    else:
      func_name_x[name] += 1

In [ ]:
print(len(func_name))
print(len(func_name_o))
print(len(func_name_x))
func_name_t = sorted(func_name.items(), key=lambda x: x[1], reverse=True)[:50]
func_name_o_t = sorted(func_name_o.items(), key=lambda x: x[1], reverse=True)[:50]
func_name_x_t = sorted(func_name_x.items(), key=lambda x: x[1], reverse=True)[:50]
print(func_name_t)
print(func_name_o_t)
print(func_name_x_t)

[('GetProcAddress', 4906), ('GetLastError', 4832), ('GetCurrentProcess', 4481), ('CloseHandle', 4473), ('Sleep', 4362), ('MultiByteToWideChar', 4234), ('GetTickCount', 4104), ('ExitProcess', 4001), ('WriteFile', 3895), ('GetCurrentThreadId', 3791), ('GetModuleHandleA', 3647), ('RegCloseKey', 3631), ('FreeLibrary', 3542), ('UnhandledExceptionFilter', 3435), ('QueryPerformanceCounter', 3434), ('GetCurrentProcessId', 3415), ('WideCharToMultiByte', 3375), ('LoadLibraryA', 3265), ('GetSystemTimeAsFileTime', 3246), ('ReadFile', 3228), ('TerminateProcess', 3227), ('SetFilePointer', 3140), ('SetUnhandledExceptionFilter', 3074), ('DeleteCriticalSection', 3067), ('LeaveCriticalSection', 3012), ('EnterCriticalSection', 3009), ('WaitForSingleObject', 2834), ('GetCommandLineA', 2720), ('GetModuleFileNameA', 2610), ('FindClose', 2559), ('GetFileSize', 2545), ('CreateFileA', 2473), ('RtlUnwind', 2447), ('InitializeCriticalSection', 2413), ('LocalFree', 2399), ('_initterm', 2394), ('GetDC', 2394), ('G

In [ ]:
# 정상
list1 = func_name_o_t
# 악성
list2 = func_name_x_t
# 정상에서 뺀 거
dic_o = {}
dic_x = {}
for i in range(len(list1)):
  for j in range(len(list2)):
    if list1[i][0] == list2[j][0]:
      break
  if j == len(list2)-1:
    dic_o[list1[i][0]] = list1[i][1]
print(dic_o)

# 악성에서 뺀 거
for i in range(len(list2)):
  for j in range(len(list1)):
    if list2[i][0] == list1[j][0]:
      break
  if j == len(list1)-1:
    dic_x[list2[i][0]] = list2[i][1]
print(dic_x)

print(len(dic_o))
print(len(dic_x))

{'QueryPerformanceCounter': 3434, 'GetCurrentProcessId': 3415, 'GetSystemTimeAsFileTime': 3246, 'TerminateProcess': 3227, 'SetUnhandledExceptionFilter': 3074, 'InitializeCriticalSection': 2413, 'LocalFree': 2399, '_initterm': 2394, 'GetDC': 2394, 'CoTaskMemFree': 2325, 'CoCreateInstance': 2308, 'free': 2281, 'GetWindowRect': 2262, 'TlsGetValue': 2260}
{'': 159559, 'RegQueryValueExA': 5448, 'RegOpenKeyExA': 5358, 'SelectObject': 5280, 'GetDeviceCaps': 5019, 'SetTextColor': 4967, 'CharNextA': 4890, 'SetBkColor': 4879, 'FindFirstFileA': 4766, 'SetBkMode': 4735, 'DestroyWindow': 4552, 'GetStartupInfoA': 4496, 'GetFileType': 4437, 'lstrcpynA': 4427, 'ImageList_Create': 4411}
14
15
15


In [ ]:
print(dic_o)
print(dic_x)
print(len(dic_o))
print(len(dic_x))

#dic_x = {'VerQueryValueA': 3826, 'GetFileVersionInfoA': 3381, 'GetFileVersionInfoSizeA': 3376, 'lstrcmpiA': 3359, 'HeapReAlloc': 3285, 'FindNextFileA': 3272, 'SetErrorMode': 3241, 'DispatchMessageA': 3241, 'IsWindowEnabled': 3200}